# COVID19 - Brazil States

# import Ray and test CUDA

In [1]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",512)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.shutdown()
ray.init(object_store_memory=1*GB,memory=1*GB,lru_evict=True,\
         driver_object_store_memory=500*MB,num_gpus=None,num_cpus=1, ignore_reinit_error=True) # , include_webui=False, ignore_reinit_error=True)

# ray.init(lru_evict=True,object_store_memory=2*GB,memory=4*GB,
#          num_gpus=312,num_cpus=10, include_webui=False, ignore_reinit_error=True)

# ray.shutdown()
# ray.init(num_cpus=10,num_gpus=None)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2020-07-04 21:39:00,126	INFO resource_spec.py:212 -- Starting Ray with 0.98 GiB memory available for workers and up to 1.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-04 21:39:00,468	INFO services.py:1170 -- View the Ray dashboard at localhost:8269


ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)

# Import Modules

In [2]:
import urllib.request
import pandas as pd
import numpy as np

# Download data

In [3]:
import get_data
LoadData=False

if LoadData:
    get_data.get_data()

In [4]:
df = pd.read_csv("data/dados_total_estados.csv")
df

(pid=138941) ray.get_gpu_ids(): 0
(pid=138941) CUDA_VISIBLE_DEVICES: 0


,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-07-04,AP,TOTAL,state,29809,441,106,True,845731.0,16.0,3524.64318,0.0148
1,1,2020-07-03,AP,TOTAL,state,29574,438,105,False,845731.0,16.0,3496.85657,0.0148
2,2,2020-07-02,AP,TOTAL,state,29153,427,104,False,845731.0,16.0,3447.07714,0.0146
3,3,2020-07-01,AP,TOTAL,state,28927,420,103,False,845731.0,16.0,3420.35470,0.0145
4,4,2020-06-30,AP,TOTAL,state,28492,417,102,False,845731.0,16.0,3368.91990,0.0146
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,262118,2020-02-29,SP,TOTAL,state,2,0,5,False,45919049.0,35.0,0.00436,0.0000
3002,262119,2020-02-28,SP,TOTAL,state,2,0,4,False,45919049.0,35.0,0.00436,0.0000
3003,262120,2020-02-27,SP,TOTAL,state,1,0,3,False,45919049.0,35.0,0.00218,0.0000
3004,262121,2020-02-26,SP,TOTAL,state,1,0,2,False,45919049.0,35.0,0.00218,0.0000


# Model

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma_2+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + d \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t) + c \cdot a(t)
\end{array}$$

$$\gamma = a + b$$

$$\gamma_2 = c + d$$

$$p=0.2$$

The last equation does not need to be solve because:

$$s + e + i + a + r + k = N$$ 

$N$ is the total population of the region, state or country. If you derivate and by considering population constant during the period of simulation, you get the equation of $k$ without depending of the $N$.

$$\frac{d k}{d t}=-(\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.

## Parameters

$\beta$: Effective contact rate [1/min]
    
$\gamma$: from infected Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$\gamma_2$: from assymptomatic Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed from infected  [1/min]

$b$: recovery rate from infected [1/min]

$c$: mortality of healed from assymptomatic [1/min]

$d$: recovery rate from assymptomatic [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

# Initial parameters

In [5]:
paramOpt=4
changeCSV=True
adjustParam=False
paramSave=False

if paramOpt==0:
    paramFile="data/param.csv"
    version = "1"
    model = "ManualIC"

if paramOpt==1:
    paramFile="data/param_optimized_DE_Scipy.csv"    
    version = "1002"
    model = "DE_ScipyIC"
    
if paramOpt==2:
    paramFile="data/param_optimized_DE_SciPy_HistMin.csv"
    version = "1002"
    model = "DE_ScipyIC"

if paramOpt==3:
    paramFile="data/param_optimized_Yabox.csv"    
    version = "104"
    model = "YaboxIC"
    
if paramOpt==4:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version = "104"
    model = "YaboxIC"

if paramOpt==5:
    paramFile="data/param_optimized_Brute_Force.csv"
    version = "12"
    model = "BruteForceIC"

if paramOpt==6:
    paramFile="data/param_optimized_Brute_Force_HistMin.csv"
    version = "12"
    model = "BruteForceIC"

dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
states=dfparam.state

if changeCSV:
    dfparam.loc[dfparam.state=='AM','d0'] = 20
    dfparam.loc[dfparam.state=='PA','d0'] = 50
    dfparam.loc[dfparam.state=='PI','d0'] = 0
    dfparam.loc[dfparam.state=='RR','d0'] = 0
    dfparam.loc[dfparam.state=='AP','d0'] = 0
    dfparam.loc[dfparam.state=='RJ','d0'] = 10
    dfparam.loc[dfparam.state=='PE','d0'] = 50
    dfparam.loc[dfparam.state=='MG','d0'] = 20
    dfparam.loc[dfparam.state=='SP','d0'] = 105
    dfparam.loc[dfparam.state=='SP','WCASES'] = 0.65
    dfparam.loc[dfparam.state=='PI','i0'] = 750
    dfparam.loc[dfparam.state=='PE','i0'] = 500
    dfparam.loc[dfparam.state=='RR','i0'] = 550
    dfparam.loc[dfparam.state=='AP','i0'] = 650

if paramSave:
    dfparam.to_csv(paramFile)    
    
if adjustParam:
    sCorrect=[1.25,1.4,1.3,1.25,1.2,1.45,1.45,1.2,1.0,1.0,1.1]
    dfparam.s0=dfparam.s0.multiply(sCorrect[:len(states)], axis=0)

dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,1.147464e+07,0,0,599,300,105,100,0.15,0.650000,0.019385
1,ES,2020-04-01,200,4.697286e+05,0,0,44,250,50,100,0.15,0.639412,0.007067
2,MG,2020-04-01,200,1.475001e+06,0,0,251,250,20,100,0.15,0.612272,0.005835
3,RJ,2020-03-20,200,8.130151e+05,0,0,624,250,10,100,0.15,0.635137,0.004899
4,CE,2020-03-20,200,6.647835e+05,0,0,512,250,50,100,0.15,0.643318,0.025334
5,PE,2020-03-20,200,9.054554e+05,0,0,500,250,50,100,0.15,0.664496,0.012130
6,AM,2020-03-20,200,8.966598e+05,0,0,563,250,20,100,0.15,0.694524,0.016627
7,PA,2020-03-20,200,8.005406e+05,0,0,929,250,50,100,0.15,0.498993,0.024140
8,PI,2020-03-20,200,8.860279e+05,0,0,750,250,0,100,0.15,0.671877,0.048074
9,RR,2020-03-20,200,1.037288e+06,0,0,550,250,0,100,0.15,0.611768,0.046058


# Define States Array and Import Solver

In [6]:
display(states)
import ray
import cupy
from numba import cuda
ncpu=len(states)
# ray.init(num_cpus=ncpu)

# Load solver
%reload_ext autoreload
%autoreload 2
import LearnerYabox_v3 as Learner #Yabox

0     SP
1     ES
2     MG
3     RJ
4     CE
5     PE
6     AM
7     PA
8     PI
9     RR
10    AP
Name: state, dtype: object

# Solve for States Array

In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [ ]:
allStates=False
cleanRecovered=False
version="20"

results=[]
if allStates:
    for state in states:        
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
        result = f.train.remote() 
        results.append(result)
else:
    state= "AM"
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    parameters
    cleanRecovered = False
    f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

  0%|          | 10011/3500000 [02:17<5:41:36, 170.28it/s]


  1%|          | 39340/3500000 [04:13<2:53:55, 331.61it/s]


  3%|▎         | 87990/3500000 [06:01<1:41:12, 561.89it/s]


  4%|▍         | 154846/3500000 [07:44<1:17:25, 720.10it/s]


  7%|▋         | 240471/3500000 [09:29<59:05, 919.40it/s]


...

 98%|█████████▊| 3428271/3500000 [34:03<00:21, 3368.36it/s]


100%|█████████▉| 3499335/3500000 [34:24<00:00, 3373.67it/s]
3501981it [34:24, 3462.75it/s]                             
3504628it [34:25, 3409.25it/s]
3507276it [34:26, 3359.80it/s]
3509925it [34:27, 3375.82it/s]
3512575it [34:28, 3326.12it/s]
3515226it [34:29, 3218.97it/s]
3517878it [34:29, 3244.34it/s]
3520531it [34:30, 3298.51it/s]
3523185it [34:31, 3371.01it/s]
3525840it [34:32, 3363.32it/s]
3528496it [34:33, 3305.73it/s]
3531153it [34:33, 3356.63it/s]
3533811it [34:34, 3433.69it/s]
3536470it [34:35, 3308.79it/s]
3539130it [34:36, 3323.33it/s]
3541791it [34:36, 3369.32it/s]
3544453it [34:37, 3396.79it/s]
3547116it [34:38, 3406.31it/s]
3549780it [34:39, 3412.98it/s]
3552445it [34:40, 3403.10it/s]
3555111it [34:40, 3409.54it/s]
3557778it [34:41, 3487.44it/s]
3560446it [34:42, 3455.31it/s]
3563115it [34:43, 3259.37it/s]
3565785it [34:44, 3289.50it/s]
3568456it [34:44, 3279.84it/s]
3571128it [34:45, 3281.06it/s]
3573801it [34:46, 3230.24it/s]
3576475it [34:47, 3302.89it/s]
3579150it [3

4085511it [37:12, 3635.80it/s]
4088370it [37:13, 3615.23it/s]
4091230it [37:13, 3659.89it/s]
4094091it [37:14, 3659.49it/s]
4096953it [37:15, 3609.45it/s]
4099816it [37:16, 3612.08it/s]
4102680it [37:16, 3686.03it/s]
4105545it [37:17, 3664.41it/s]
4108411it [37:18, 3593.91it/s]
4111278it [37:19, 3572.36it/s]
4114146it [37:20, 3549.19it/s]
4117015it [37:21, 3523.07it/s]
4119885it [37:21, 3481.43it/s]
4122756it [37:22, 3393.16it/s]
4125628it [37:23, 3411.86it/s]
4128501it [37:24, 3413.43it/s]
4131375it [37:25, 3471.72it/s]
4134250it [37:26, 3420.22it/s]
4137126it [37:26, 3539.29it/s]
4140003it [37:27, 3512.51it/s]
4142881it [37:28, 3504.12it/s]
4145760it [37:29, 3614.18it/s]
4148640it [37:30, 3526.11it/s]
4151521it [37:30, 3498.55it/s]
4154403it [37:31, 3450.92it/s]
4157286it [37:32, 3353.25it/s]
4160170it [37:33, 3349.02it/s]
4163055it [37:34, 3396.27it/s]
4165941it [37:35, 3331.64it/s]
4168828it [37:36, 3357.67it/s]
4171716it [37:37, 3339.70it/s]
4174605it [37:37, 3323.36it/s]
4177495i

4878126it [41:20, 2943.86it/s]
4881250it [41:21, 2963.30it/s]
4884375it [41:22, 2972.01it/s]
4887501it [41:23, 2945.69it/s]
4890628it [41:24, 2941.81it/s]
4893756it [41:26, 2868.22it/s]
4896885it [41:27, 2914.37it/s]
4900015it [41:28, 2956.76it/s]
4903146it [41:29, 2854.27it/s]
4906278it [41:30, 2990.87it/s]
4909411it [41:31, 2963.86it/s]
4912545it [41:32, 2919.64it/s]
4915680it [41:33, 2971.75it/s]
4918816it [41:34, 2966.48it/s]
4921953it [41:35, 2954.71it/s]
4925091it [41:36, 2936.81it/s]
4928230it [41:37, 2944.15it/s]
4931370it [41:38, 3030.12it/s]
4934511it [41:39, 2967.03it/s]
4937653it [41:40, 2962.62it/s]
4940796it [41:41, 2944.08it/s]
4943940it [41:43, 2877.25it/s]
4947085it [41:44, 2859.66it/s]
4950231it [41:45, 2809.94it/s]
4953378it [41:46, 2936.82it/s]
4956526it [41:47, 2951.20it/s]
4959675it [41:48, 2861.68it/s]
4962825it [41:49, 2876.01it/s]
4965976it [41:50, 2900.82it/s]
4969128it [41:51, 2922.24it/s]
4972281it [41:52, 2904.23it/s]
4975435it [41:53, 2946.63it/s]
4978590i

5740966it [46:22, 2936.87it/s]
5744355it [46:23, 2902.61it/s]
5747745it [46:25, 2899.87it/s]
5751136it [46:26, 2910.99it/s]
5754528it [46:27, 2942.42it/s]
5757921it [46:28, 2957.78it/s]
5761315it [46:29, 2979.99it/s]
5764710it [46:30, 3034.59it/s]
5768106it [46:31, 3095.95it/s]
5771503it [46:32, 3062.24it/s]
5774901it [46:34, 3044.69it/s]
5778300it [46:35, 3014.65it/s]
5781700it [46:36, 2950.42it/s]
5785101it [46:37, 2919.74it/s]
5788503it [46:38, 2891.87it/s]
5791906it [46:39, 2948.31it/s]
5795310it [46:41, 2875.64it/s]
5798715it [46:42, 2886.48it/s]
5802121it [46:43, 2961.33it/s]
5805528it [46:44, 3000.47it/s]
5808936it [46:45, 3001.22it/s]
5812345it [46:46, 2988.03it/s]
5815755it [46:47, 2976.23it/s]
5819166it [46:49, 2969.44it/s]
5822578it [46:50, 2958.27it/s]
5825991it [46:51, 2932.77it/s]
5829405it [46:52, 2966.59it/s]
5832820it [46:53, 2904.54it/s]
5836236it [46:54, 2880.72it/s]
5839653it [46:56, 2858.59it/s]
5843071it [46:57, 2953.60it/s]
5846490it [46:58, 2949.22it/s]
5849910i

In [ ]:
ray.shutdown()

# Plots

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#select districts for plotting log plot
states4Plot=['MG',
               'SP',
               'ES',
               'CE',
               'RJ']

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#number of cases to start plotting model in log graph - real data = 100
startCase=100

In [ ]:
#plots one district or all districts
from datetime import datetime,timedelta
dfparam = pd.read_csv(paramFile)
plotStates=False

#select states for plotting
#you need to run (solve) it before plotting
if plotStates:
    states=dfparam.state
    allStates=True

#main state for plotting log graph or SEAIRD plot
state="SP"

#if it is not SEAIRD plot, so plot one state only
if not opt==5:
    AllStates=False

if allStates:
    for state in states:
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        ratio = query['RATIO'][0]
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        
        #calcula data máxima dos gráficos
        #100 dias é usado como máximo dos cálculos da derivada das mortes
        lastDate=df.date.max()
        maxDate= datetime.strptime(lastDate, '%Y-%m-%d') + timedelta(days = 100) #"2020-08-31"
        maxDateStr = maxDate.strftime("%Y-%m-%d")

        covid_plots.covid_plots(state, states4Plot, startdate, predict_range, \
                        startCase, 5, version, show=True, ratio=ratio, maxDate=maxDateStr,model=model)
else: 
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    ratio = query['RATIO'][0]
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    
    #calcula data máxima dos gráficos
    #100 dias é usado como máximo dos cálculos da derivada das mortes
    lastDate=df.date.max()
    maxDate= datetime.strptime(lastDate, '%Y-%m-%d') + timedelta(days = 100) #"2020-08-31"
    maxDateStr = maxDate.strftime("%Y-%m-%d")    
    
    covid_plots.covid_plots(state, states4Plot, startdate,predict_range, \
                       startCase, opt, version, show=True, ratio=ratio, maxDate=maxDateStr,model=model)